In [12]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import utils
import plot_utils

# Read data

In [13]:
data_file = './data/paleo_meso_ceno_data_period.csv'
data = pd.read_csv(data_file)
data = data.sort_values(by=['mean_ma'], ascending=False)

time_file = './data/timeScale.xlsx'
time = pd.read_excel(time_file,sheet_name='Sheet1')
time_period = utils.get_timesubset(time,3,541)
width_period = [time_period['max_ma'][i] - time_period['min_ma'][i] for i in list(time_period.index)]

# Skeletal abundance

In [14]:
data_skeletal = utils.get_stats(data,'skeletal_total','period')
data_skeletal = data_skeletal.sort_values(by=['time'], ascending=True)

## Reef data

In [15]:
# reef only data
reefs =['patchReefs', 'reef','interReef','mudMound','foreReef', 'bioherm', 'patchReef','backReef']
reef_data = data[data['depo_env2'].isin(reefs)].copy()
non_reef_data = data[~data['depo_env2'].isin(reefs)].copy()

In [16]:
quart_reef = reef_data[reef_data.period == 'Quaternary']
reef_data = reef_data[~(reef_data.period == 'Quaternary')]

quart_non_reef = non_reef_data[non_reef_data.period == 'Quaternary']
non_reef_data = non_reef_data[~(non_reef_data.period == 'Quaternary')]

In [17]:
data_skeletal_reef = utils.get_stats(reef_data,'skeletal_total','period')
data_skeletal_reef = data_skeletal_reef.sort_values(by=['time'], ascending=True)

## Paleocontinent data

In [18]:
paleocontinents = data['Paleocontinents1'].dropna().unique()

In [19]:
paleocontinents.sort()

In [20]:
data_skeletal_continent = []
colors = ["#FFFFFF","#004949","#009292","#ff6db6","#ffb6db","#252525",
          "#490092","#006ddb","#b66dff","#6db6ff","#b6dbff",
          "#920000","#924900","#db6d00","#24ff24","#ffff6d"]
for i, continent in enumerate(paleocontinents):
    subset = data[(data['Paleocontinents1'] == continent)]
    data_skeletal_subset = utils.get_stats(subset,'skeletal_total','period')
    data_skeletal_subset = data_skeletal_subset.sort_values(by=['time'], ascending=True)
    data_skeletal_subset['paleocontinent'] = continent
    data_skeletal_subset['color'] = colors[i]
    data_skeletal_continent.append(data_skeletal_subset)
data_skeletal_continent = pd.concat(data_skeletal_continent)

# Plot Figure 2

In [21]:
fig = make_subplots(rows=3, cols=2,column_widths=[0.9,0.1],
                    row_heights = [0.46, 0.46, 0.08],
                    specs=[[{"secondary_y": False},{"secondary_y": False}],
                           [{"secondary_y": False},{"secondary_y": False}],
                           [{"secondary_y": False},{"secondary_y": False}],
                          ],
                    shared_yaxes = True,shared_xaxes = True,vertical_spacing = 0.001,horizontal_spacing=0.009)

#Row 1
# reef and non reef

fig.add_trace(go.Scatter(x = data_skeletal['time'][1:],y = data_skeletal['mean'][1:],mode = 'lines+markers',
                         name = 'All data mean', showlegend = True,legendgroup='1',
                         marker = dict(color = 'black',size = 8),
                         ),row=1,col=1)
fig.add_trace(go.Scatter(x = data_skeletal['time'][:1],y = data_skeletal['mean'][:1],mode = 'lines+markers',
                         name = 'All data Mean', showlegend = False, 
                         marker = dict(color = 'black',size = 8),
                         ),row=1,col=2)

fig.add_trace(go.Box(x = reef_data['mean_ma']-5,y = reef_data['skeletal_total'],#mode = 'lines+markers',
                         name = 'Reef', showlegend = True,width=8,boxmean=True,legendgroup='1',
                         marker = dict(color = 'red',size = 3,),
                         ),row=1,col=1) ## -5 is for positioning the box on the plot
fig.add_trace(go.Box(x = non_reef_data['mean_ma']+5,y = non_reef_data['skeletal_total'],#mode = 'lines+markers',
                         name = 'Non-reef', showlegend = True,width=8,boxmean=True,legendgroup='1',
                         marker = dict(color = 'blue',size = 3,),
                         ),row=1,col=1) ## +5 is for positioning the box on the plot

fig.add_trace(go.Box(x = quart_reef['mean_ma']-0.7,y = quart_reef['skeletal_total'],#mode = 'lines+markers',
                         name = 'Reef', showlegend = False,width=0.5,boxmean=True,
                         marker = dict(color = 'red',size = 0.05,),
                         ),row=1,col=2) ## -5 is for positioning the box on the plot
fig.add_trace(go.Box(x = quart_non_reef['mean_ma']+0.7,y = quart_non_reef['skeletal_total'],#mode = 'lines+markers',
                         name = 'Non-reef', showlegend = False,width=0.5,boxmean=True,
                         marker = dict(color = 'blue',size = 0.05,),
                         ),row=1,col=2) ## +5 is for positioning the box on the plot

fig.update_xaxes(range=(540,2.01),ticks = 'outside',showline = True, showticklabels = False,mirror=True,
                          linecolor = 'black',row=1,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',showline = True, showticklabels = False,mirror=True,
                          linecolor = 'black',row=1,col=2)

fig.update_layout(boxmode='group')

# Row 2

fig.add_trace(go.Scatter(x = data_skeletal['time'][1:],y = data_skeletal['mean'][1:],mode = 'lines+markers',
                         name = 'All data mean', 
                         showlegend = True, legendgroup='2',
                         marker = dict(color = 'black',size = 8)
                         ),row=2,col=1)

fig.add_trace(go.Scatter(x = data_skeletal['time'][:1],y = data_skeletal['mean'][:1],mode = 'lines+markers',
                         
                         showlegend = False,
                         marker = dict(color = 'black',size = 8)
                         ),row=2,col=2)

for i, continent in enumerate(paleocontinents):
    subset = data_skeletal_continent[data_skeletal_continent['paleocontinent'] == continent]
    
    fig.add_trace(go.Scatter(x = subset[~(subset['period'] =='Quaternary')]['time'],
                             y = subset[~(subset['period']=='Quaternary')]['mean'],mode = 'markers',
                             showlegend = True, legendgroup='2', name=continent,
                             marker = dict(size = 15, color=colors[i], opacity=0.8,
                                           symbol='triangle-up', line=dict(color='black',width=1)),
                             ),row=2,col=1)

    if len(subset[~(subset['period'] =='Quaternary')]) == 0:
        showlegend=True
    else:
        showlegend=False
    fig.add_trace(go.Scatter(x = subset[subset['period']=='Quaternary']['time'],
                             y = subset[subset['period']=='Quaternary']['mean'],mode = 'markers',
                             showlegend = showlegend, legendgroup='2', name=continent,
                             marker = dict(size = 15, color=colors[i], opacity=0.8,
                                           symbol='triangle-up', line=dict(color='black',width=1)),
                             ),row=2,col=2)

fig.update_xaxes(ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=2,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',showline = True, 
                         mirror = True,linecolor = 'black',row=2,col=2)

#Row 3
#time
plot_utils.plot_time(fig,time_period[1:],width_period[1:],3,1)
plot_utils.plot_time(fig,time_period[:1],width_period[:1],3,2)

#y_axes
fig.update_yaxes(title = 'Skeletal biomass %',ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=1,col=1)
fig.update_yaxes(title = 'Skeletal biomass %',ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=2,col=1)
fig.update_yaxes(ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=1,col=2)
fig.update_yaxes(ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=2,col=2)


#x_axes
fig.update_xaxes(range=(540,2.01),ticks = 'outside',title = 'Time (Ma)',row=3,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',title = 'Time (Ma)',row=3,col=2)




fig.update_layout(height = 1000, width = 900,font_color = 'black', font_family='Helvetica',font_size=14,
                  paper_bgcolor = 'white',
                  plot_bgcolor = 'white',
                  yaxis1=dict(range=(-5,105)),
                  yaxis3=dict(range=(-5,71)),
                  yaxis5=dict(range=(0,0.2)),


                  
                  legend=dict(bgcolor='rgba(0,0,0,0)',valign='top',
                              yanchor="top",
                              y=0.99,
                              xanchor="left",
                              x=1.01),
                  legend_tracegroupgap = 310
                 )

fig.show()
fig.write_image('./figures/Figure_2.png',scale = 6)